In [1]:
import pandas as pd

dfTrain = pd.read_csv('training.csv', sep=';')
dfTest = pd.read_csv('testing.csv', sep=';')
dfIDs = pd.read_csv('task.csv')

dfIDs = dfIDs[["Name_file","rotulo"]]
dfIDs = dfIDs.rename(columns={"Name_file": "file", "rotulo": "target"})
#dfIDs

In [2]:
dfTrain = dfTrain.T
dfTrain = dfTrain.reset_index()
dfTrain.drop('index', axis = 1, inplace=True)
#dfTrain

In [3]:
dfTest = dfTest.T
dfTest = dfTest.reset_index()
dfTest.drop('index', axis = 1, inplace=True)
#dfTest

In [4]:
import cv2
import os
import numpy as np

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from tensorflow.keras.utils import to_categorical

folder = "/Fungos/"

def buildSet(indexes):
    x = []
    y = []
    for j in indexes:
        i = j-1
        img = dfIDs.loc[i, 'file']
        c = dfIDs.loc[i, 'target']
        if c == 1:
            file = 'c'+str(c)+'_'+img
            path = os.path.join(folder, file)
            mat = cv2.imread(path)
            if mat is None:
                file = 'c'+str(c+1)+'_'+img
                path = os.path.join(folder, file)
                mat = cv2.imread(path)
        else:
            file = 'c'+str(c+1)+'_'+img
            path = os.path.join(folder, file)
            mat = cv2.imread(path)
        
        mRows, mCols, mType = mat.shape
        
        if mRows>mCols:
            mat = cv2.rotate(mat, cv2.ROTATE_90_CLOCKWISE)

        mat = cv2.resize(mat, (224, 224), interpolation = cv2.INTER_AREA)
    
        x.append(mat)
        y.append(dfIDs.loc[i, 'target']-1)

    y = to_categorical(y, num_classes=3)
    x = np.array(x)
    y = np.array(y)
    return x, y

In [5]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Dropout, Flatten, Dense
from tensorflow.keras.applications import VGG16

def buildVGG(shape = (224, 224, 3), output = 3):
    classifier=tf.keras.Sequential() 
    
    vggModel= VGG16(input_shape=shape, include_top=False)
    vggModel.trainable = False
    classifier.add(vggModel)

    classifier.add(Flatten()) 
    classifier.add(Dense(4096,activation = 'relu'))
    classifier.add(BatchNormalization())
    classifier.add(Dropout(0.2, input_shape=(4096,)))
    classifier.add(Dense(2048,activation = 'relu'))
    classifier.add(BatchNormalization())
    classifier.add(Dropout(0.2, input_shape=(2048,)))

    classifier.add(Dense(output,activation = 'softmax'))

    classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
    classifier.summary()
    return classifier

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

import time

checkpoint_filepath = '/Fungos/tmp/checkpoint/VGG/'

classes = []
targets = []

for index, train in dfTrain.iterrows():
    if (index%10)==0:
        model = buildVGG()
    
    train = train.dropna()
    test = dfTest.loc[index]
    test = test.dropna()
    
    xTrain, yTrain = buildSet(train)
    xTest, yTest = buildSet(test)
    
    bs = 1
    steps = len(xTrain)//bs
    
    start_time = time.time()
    
    model_checkpoint_callback = ModelCheckpoint(
        filepath=checkpoint_filepath,
        use_multiprocessing=True,
        save_weights_only=True,
        monitor='categorical_accuracy',
        mode='max',
        save_best_only=True)
    
    history = model.fit(ImageDataGenerator().flow(xTrain, yTrain, batch_size=bs),
         callbacks=[model_checkpoint_callback],
         steps_per_epoch=steps // 32, epochs=32)
    
    print("VGG took %s seconds." % ((time.time() - start_time)))

    predictions = model.predict(xTest)
    pred = np.argmax(predictions, axis=-1)
    y = np.argmax(yTest, axis=-1)
    
    targets.append(y)
    classes.append(pred)

VGG = pd.DataFrame(classes)
Targets = pd.DataFrame(targets)

VGG.to_csv('VGG.csv', index=False)
Targets.to_csv('Targets.csv', index=False)
VGG

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 4096)              102764544 
_________________________________________________________________
batch_normalization (BatchNo (None, 4096)              16384     
_________________________________________________________________
dropout (Dropout)            (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              8390656   
_________________________________________________________________
batch_normalization_1 (Batch (None, 2048)              8

15/15 [==============================] - 8s 509ms/step - loss: 1.0317 - categorical_accuracy: 0.5333
Epoch 3/32
15/15 [==============================] - 8s 515ms/step - loss: 1.1914 - categorical_accuracy: 0.4000
Epoch 4/32
15/15 [==============================] - 8s 531ms/step - loss: 0.8929 - categorical_accuracy: 0.6667
Epoch 5/32
15/15 [==============================] - 8s 521ms/step - loss: 0.9361 - categorical_accuracy: 0.6000
Epoch 6/32
15/15 [==============================] - 8s 523ms/step - loss: 0.7247 - categorical_accuracy: 0.8000
Epoch 7/32
15/15 [==============================] - 8s 536ms/step - loss: 0.9861 - categorical_accuracy: 0.6000
Epoch 8/32
15/15 [==============================] - 8s 526ms/step - loss: 0.8595 - categorical_accuracy: 0.6667
Epoch 9/32
15/15 [==============================] - 8s 533ms/step - loss: 0.7903 - categorical_accuracy: 0.7333
Epoch 10/32
15/15 [==============================] - 8s 533ms/step - loss: 0.7732 - categorical_accuracy: 0.7333
Ep

15/15 [==============================] - 11s 751ms/step - loss: 0.7480 - categorical_accuracy: 0.8000
Epoch 19/32
15/15 [==============================] - 10s 643ms/step - loss: 0.9789 - categorical_accuracy: 0.6000
Epoch 20/32
15/15 [==============================] - 9s 620ms/step - loss: 1.2340 - categorical_accuracy: 0.3333
Epoch 21/32
15/15 [==============================] - 9s 622ms/step - loss: 1.0305 - categorical_accuracy: 0.5333
Epoch 22/32
15/15 [==============================] - 10s 671ms/step - loss: 0.9044 - categorical_accuracy: 0.6667
Epoch 23/32
15/15 [==============================] - 10s 688ms/step - loss: 1.1302 - categorical_accuracy: 0.4000
Epoch 24/32
15/15 [==============================] - 10s 639ms/step - loss: 0.9423 - categorical_accuracy: 0.6000
Epoch 25/32
15/15 [==============================] - 10s 642ms/step - loss: 0.9147 - categorical_accuracy: 0.6667
Epoch 26/32
15/15 [==============================] - 9s 576ms/step - loss: 0.8866 - categorical_accura

15/15 [==============================] - 8s 536ms/step - loss: 0.7539 - categorical_accuracy: 0.8000
Epoch 21/32
15/15 [==============================] - 8s 527ms/step - loss: 1.2006 - categorical_accuracy: 0.4000
Epoch 22/32
15/15 [==============================] - 8s 525ms/step - loss: 0.8707 - categorical_accuracy: 0.6667
Epoch 23/32
15/15 [==============================] - 8s 536ms/step - loss: 1.1285 - categorical_accuracy: 0.4667
Epoch 24/32
15/15 [==============================] - 8s 534ms/step - loss: 1.0055 - categorical_accuracy: 0.5333
Epoch 25/32
15/15 [==============================] - 8s 535ms/step - loss: 0.7090 - categorical_accuracy: 0.8667
Epoch 26/32
15/15 [==============================] - 8s 553ms/step - loss: 0.8157 - categorical_accuracy: 0.7333
Epoch 27/32
15/15 [==============================] - 8s 537ms/step - loss: 1.0054 - categorical_accuracy: 0.6000
Epoch 28/32
15/15 [==============================] - 8s 546ms/step - loss: 0.8701 - categorical_accuracy: 0.

15/15 [==============================] - 8s 502ms/step - loss: 0.9627 - categorical_accuracy: 0.6000
Epoch 4/32
15/15 [==============================] - 8s 509ms/step - loss: 0.7176 - categorical_accuracy: 0.8000
Epoch 5/32
15/15 [==============================] - 8s 505ms/step - loss: 0.9545 - categorical_accuracy: 0.6000
Epoch 6/32
15/15 [==============================] - 8s 505ms/step - loss: 1.3169 - categorical_accuracy: 0.3333
Epoch 7/32
15/15 [==============================] - 8s 512ms/step - loss: 0.8859 - categorical_accuracy: 0.6667
Epoch 8/32
15/15 [==============================] - 8s 537ms/step - loss: 1.0177 - categorical_accuracy: 0.5333
Epoch 9/32
15/15 [==============================] - 8s 520ms/step - loss: 0.9723 - categorical_accuracy: 0.6000
Epoch 10/32
15/15 [==============================] - 8s 511ms/step - loss: 1.0280 - categorical_accuracy: 0.5333
Epoch 11/32
15/15 [==============================] - 8s 516ms/step - loss: 0.9637 - categorical_accuracy: 0.6000
E

...

15/15 [==============================] - 7s 480ms/step - loss: 1.0223 - categorical_accuracy: 0.5333
Epoch 7/32
15/15 [==============================] - 7s 477ms/step - loss: 0.9597 - categorical_accuracy: 0.6000
Epoch 8/32
15/15 [==============================] - 7s 485ms/step - loss: 0.9650 - categorical_accuracy: 0.6000
Epoch 9/32
15/15 [==============================] - 7s 482ms/step - loss: 0.8202 - categorical_accuracy: 0.7333
Epoch 10/32
15/15 [==============================] - 7s 489ms/step - loss: 0.8817 - categorical_accuracy: 0.6667
Epoch 11/32
15/15 [==============================] - 7s 485ms/step - loss: 1.2027 - categorical_accuracy: 0.4000
Epoch 12/32
15/15 [==============================] - 7s 492ms/step - loss: 0.9014 - categorical_accuracy: 0.6667
Epoch 13/32
15/15 [==============================] - 7s 483ms/step - loss: 0.9696 - categorical_accuracy: 0.6000
Epoch 14/32
15/15 [==============================] - 7s 488ms/step - loss: 1.1647 - categorical_accuracy: 0.400

15/15 [==============================] - 7s 478ms/step - loss: 0.8792 - categorical_accuracy: 0.6599
Epoch 9/32
15/15 [==============================] - 7s 478ms/step - loss: 0.9139 - categorical_accuracy: 0.6469
Epoch 10/32
15/15 [==============================] - 7s 478ms/step - loss: 0.8775 - categorical_accuracy: 0.6911
Epoch 11/32
15/15 [==============================] - 7s 479ms/step - loss: 0.9995 - categorical_accuracy: 0.5648
Epoch 12/32
15/15 [==============================] - 7s 488ms/step - loss: 1.0952 - categorical_accuracy: 0.5008
Epoch 13/32
15/15 [==============================] - 7s 484ms/step - loss: 0.7601 - categorical_accuracy: 0.7884
Epoch 14/32
15/15 [==============================] - 7s 479ms/step - loss: 1.0959 - categorical_accuracy: 0.4823
Epoch 15/32
15/15 [==============================] - 7s 471ms/step - loss: 0.7942 - categorical_accuracy: 0.7749
Epoch 16/32
15/15 [==============================] - 7s 485ms/step - loss: 0.8767 - categorical_accuracy: 0.6

15/15 [==============================] - 7s 479ms/step - loss: 1.0780 - categorical_accuracy: 0.4667
Epoch 25/32
15/15 [==============================] - 8s 503ms/step - loss: 1.0158 - categorical_accuracy: 0.5333
Epoch 26/32
15/15 [==============================] - 7s 481ms/step - loss: 1.1163 - categorical_accuracy: 0.4000
Epoch 27/32
15/15 [==============================] - 7s 485ms/step - loss: 0.8761 - categorical_accuracy: 0.7333
Epoch 28/32
15/15 [==============================] - 7s 482ms/step - loss: 1.1484 - categorical_accuracy: 0.4000
Epoch 29/32
15/15 [==============================] - 8s 505ms/step - loss: 1.0528 - categorical_accuracy: 0.5333
Epoch 30/32
15/15 [==============================] - 7s 481ms/step - loss: 1.0223 - categorical_accuracy: 0.5333
Epoch 31/32
15/15 [==============================] - 7s 494ms/step - loss: 0.9735 - categorical_accuracy: 0.6000
Epoch 32/32
15/15 [==============================] - 7s 479ms/step - loss: 0.8520 - categorical_accuracy: 0.

15/15 [==============================] - 7s 474ms/step - loss: 1.1894 - categorical_accuracy: 0.4667
Epoch 8/32
15/15 [==============================] - 7s 475ms/step - loss: 1.0564 - categorical_accuracy: 0.5333
Epoch 9/32
15/15 [==============================] - 7s 471ms/step - loss: 1.1024 - categorical_accuracy: 0.4667
Epoch 10/32
15/15 [==============================] - 7s 479ms/step - loss: 1.1221 - categorical_accuracy: 0.4000
Epoch 11/32
15/15 [==============================] - 7s 477ms/step - loss: 1.0060 - categorical_accuracy: 0.6000
Epoch 12/32
15/15 [==============================] - 7s 473ms/step - loss: 0.9741 - categorical_accuracy: 0.6000
Epoch 13/32
15/15 [==============================] - 7s 475ms/step - loss: 0.8178 - categorical_accuracy: 0.7333
Epoch 14/32
15/15 [==============================] - 7s 484ms/step - loss: 0.8125 - categorical_accuracy: 0.7333
Epoch 15/32
15/15 [==============================] - 7s 476ms/step - loss: 0.6518 - categorical_accuracy: 0.80

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,2,1,1,1,0,2,1,0,1,1,...,1,1,0,0,2,2,0,2.0,1.0,2.0
1,1,1,1,1,1,2,2,2,2,0,...,2,2,2,0,0,0,2,0.0,NaN,NaN
2,1,1,1,2,2,1,1,2,1,2,...,2,2,2,2,2,2,2,0.0,2.0,NaN
3,1,1,1,1,1,1,1,1,1,1,...,2,0,2,2,2,2,2,0.0,2.0,NaN
4,1,2,1,1,1,1,1,2,2,1,...,2,2,2,2,2,2,2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,2,0,0,0,0,0,0,0,2,...,2,0,0,0,0,2,2,2.0,2.0,NaN
96,2,0,0,0,0,2,2,0,0,0,...,1,0,0,0,2,0,2,2.0,2.0,NaN
97,0,2,0,2,2,2,2,2,2,2,...,2,2,2,2,2,0,0,2.0,0.0,NaN
98,2,2,2,2,2,2,0,0,2,2,...,2,2,2,2,2,0,0,2.0,0.0,NaN


In [6]:
Targets

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2.0,2.0,2.0
1,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2.0,NaN,NaN
2,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2.0,2.0,NaN
3,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2.0,2.0,NaN
4,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2.0,2.0,NaN
96,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2.0,2.0,NaN
97,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2.0,2.0,NaN
98,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2.0,2.0,NaN


In [7]:
checkpoint_filepath = '/Fungos/tmp/checkpoint/VGGDA/'

classes = []

for index, train in dfTrain.iterrows():
    if (index%10)==0:
        model = buildVGG()
    
    train = train.dropna()
    test = dfTest.loc[index]
    test = test.dropna()
    
    xTrain, yTrain = buildSet(train)
    xTest, yTest = buildSet(test)
    
    datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True)
    
    datagen.fit(xTrain)
    
    bs = 1
    steps = len(xTrain)//bs
    
    start_time = time.time()
    
    model_checkpoint_callback = ModelCheckpoint(
        filepath=checkpoint_filepath,
        use_multiprocessing=True,
        save_weights_only=True,
        monitor='categorical_accuracy',
        mode='max',
        save_best_only=True)
    
    history = model.fit(datagen.flow(xTrain, yTrain, batch_size=bs),
         callbacks=[model_checkpoint_callback],
         steps_per_epoch=steps // 32, epochs=32)
    
    print("VGG took %s seconds." % ((time.time() - start_time)))

    predictions = model.predict(xTest)
    pred = np.argmax(predictions, axis=-1)
    
    classes.append(pred)

VGGDA = pd.DataFrame(classes)

VGGDA.to_csv('VGGDA.csv', index=False)
VGGDA

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_10 (Flatten)         (None, 25088)             0         
_________________________________________________________________
dense_30 (Dense)             (None, 4096)              102764544 
_________________________________________________________________
batch_normalization_20 (Batc (None, 4096)              16384     
_________________________________________________________________
dropout_20 (Dropout)         (None, 4096)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 2048)              8390656   
_________________________________________________________________
batch_normalization_21 (Batc (None, 2048)            

15/15 [==============================] - 8s 478ms/step - loss: 1.0211 - categorical_accuracy: 0.5333
Epoch 3/32
15/15 [==============================] - 7s 477ms/step - loss: 1.1160 - categorical_accuracy: 0.4667
Epoch 4/32
15/15 [==============================] - 7s 472ms/step - loss: 0.9080 - categorical_accuracy: 0.6667
Epoch 5/32
15/15 [==============================] - 7s 477ms/step - loss: 0.7342 - categorical_accuracy: 0.8000
Epoch 6/32
15/15 [==============================] - 7s 483ms/step - loss: 0.7870 - categorical_accuracy: 0.7333
Epoch 7/32
15/15 [==============================] - 7s 479ms/step - loss: 1.0845 - categorical_accuracy: 0.5333
Epoch 8/32
15/15 [==============================] - 7s 488ms/step - loss: 0.8011 - categorical_accuracy: 0.7333
Epoch 9/32
15/15 [==============================] - 7s 483ms/step - loss: 0.6807 - categorical_accuracy: 0.8000
Epoch 10/32
15/15 [==============================] - 7s 487ms/step - loss: 1.1139 - categorical_accuracy: 0.5333
Ep

15/15 [==============================] - 7s 489ms/step - loss: 1.2209 - categorical_accuracy: 0.2667
Epoch 19/32
15/15 [==============================] - 7s 497ms/step - loss: 0.9805 - categorical_accuracy: 0.6000
Epoch 20/32
15/15 [==============================] - 7s 487ms/step - loss: 0.8710 - categorical_accuracy: 0.7333
Epoch 21/32
15/15 [==============================] - 7s 492ms/step - loss: 0.9502 - categorical_accuracy: 0.6000
Epoch 22/32
15/15 [==============================] - 7s 486ms/step - loss: 0.8725 - categorical_accuracy: 0.6667
Epoch 23/32
15/15 [==============================] - 7s 489ms/step - loss: 0.8683 - categorical_accuracy: 0.6667
Epoch 24/32
15/15 [==============================] - 7s 488ms/step - loss: 0.8840 - categorical_accuracy: 0.6667
Epoch 25/32
15/15 [==============================] - 7s 489ms/step - loss: 0.8624 - categorical_accuracy: 0.6667
Epoch 26/32
15/15 [==============================] - 7s 486ms/step - loss: 0.7569 - categorical_accuracy: 0.

15/15 [==============================] - 7s 489ms/step - loss: 0.9884 - categorical_accuracy: 0.6000
Epoch 21/32
15/15 [==============================] - 7s 485ms/step - loss: 1.1863 - categorical_accuracy: 0.4000
Epoch 22/32
15/15 [==============================] - 8s 508ms/step - loss: 0.9086 - categorical_accuracy: 0.6667
Epoch 23/32
15/15 [==============================] - 8s 504ms/step - loss: 0.8402 - categorical_accuracy: 0.7333
Epoch 24/32
15/15 [==============================] - 7s 490ms/step - loss: 1.0298 - categorical_accuracy: 0.5333
Epoch 25/32
15/15 [==============================] - 7s 492ms/step - loss: 0.9527 - categorical_accuracy: 0.6000
Epoch 26/32
15/15 [==============================] - 7s 489ms/step - loss: 0.9696 - categorical_accuracy: 0.6000
Epoch 27/32
15/15 [==============================] - 7s 498ms/step - loss: 1.0949 - categorical_accuracy: 0.4667
Epoch 28/32
15/15 [==============================] - 7s 489ms/step - loss: 1.2868 - categorical_accuracy: 0.

15/15 [==============================] - 7s 479ms/step - loss: 1.0394 - categorical_accuracy: 0.5333
Epoch 4/32
15/15 [==============================] - 7s 485ms/step - loss: 1.0918 - categorical_accuracy: 0.4667
Epoch 5/32
15/15 [==============================] - 7s 481ms/step - loss: 1.0152 - categorical_accuracy: 0.5333
Epoch 6/32
15/15 [==============================] - 7s 477ms/step - loss: 1.0104 - categorical_accuracy: 0.5333
Epoch 7/32
15/15 [==============================] - 7s 490ms/step - loss: 0.9056 - categorical_accuracy: 0.6667
Epoch 8/32
15/15 [==============================] - 7s 484ms/step - loss: 1.1043 - categorical_accuracy: 0.4667
Epoch 9/32
15/15 [==============================] - 7s 485ms/step - loss: 1.0229 - categorical_accuracy: 0.5333
Epoch 10/32
15/15 [==============================] - 7s 488ms/step - loss: 0.8445 - categorical_accuracy: 0.7333
Epoch 11/32
15/15 [==============================] - 7s 487ms/step - loss: 0.9441 - categorical_accuracy: 0.6000
E

...

15/15 [==============================] - 11s 751ms/step - loss: 0.9145 - categorical_accuracy: 0.6667
Epoch 6/32
15/15 [==============================] - 10s 684ms/step - loss: 1.0574 - categorical_accuracy: 0.5333
Epoch 7/32
15/15 [==============================] - 12s 779ms/step - loss: 0.8873 - categorical_accuracy: 0.6667
Epoch 8/32
15/15 [==============================] - 14s 915ms/step - loss: 0.8790 - categorical_accuracy: 0.6667
Epoch 9/32
15/15 [==============================] - 15s 1s/step - loss: 1.1079 - categorical_accuracy: 0.4667
Epoch 10/32
15/15 [==============================] - 18s 1s/step - loss: 1.0245 - categorical_accuracy: 0.5333
Epoch 11/32
15/15 [==============================] - 18s 1s/step - loss: 0.8988 - categorical_accuracy: 0.6667
Epoch 12/32
15/15 [==============================] - 18s 1s/step - loss: 0.8095 - categorical_accuracy: 0.7333
Epoch 13/32
15/15 [==============================] - 18s 1s/step - loss: 1.0693 - categorical_accuracy: 0.5333
Epoch

15/15 [==============================] - 8s 507ms/step - loss: 1.0288 - categorical_accuracy: 0.5333
Epoch 14/32
15/15 [==============================] - 8s 516ms/step - loss: 0.9569 - categorical_accuracy: 0.6000
Epoch 15/32
15/15 [==============================] - 8s 523ms/step - loss: 1.1705 - categorical_accuracy: 0.4000
Epoch 16/32
15/15 [==============================] - 8s 504ms/step - loss: 1.0241 - categorical_accuracy: 0.5333
Epoch 17/32
15/15 [==============================] - 8s 500ms/step - loss: 0.9190 - categorical_accuracy: 0.6667
Epoch 18/32
15/15 [==============================] - 8s 503ms/step - loss: 0.8960 - categorical_accuracy: 0.6667
Epoch 19/32
15/15 [==============================] - 8s 509ms/step - loss: 1.0099 - categorical_accuracy: 0.5333
Epoch 20/32
15/15 [==============================] - 8s 502ms/step - loss: 0.8875 - categorical_accuracy: 0.6667
Epoch 21/32
15/15 [==============================] - 8s 512ms/step - loss: 1.0503 - categorical_accuracy: 0.

15/15 [==============================] - 8s 556ms/step - loss: 1.0454 - categorical_accuracy: 0.5333
Epoch 16/32
15/15 [==============================] - 8s 551ms/step - loss: 0.8009 - categorical_accuracy: 0.7333
Epoch 17/32
15/15 [==============================] - 9s 592ms/step - loss: 0.7202 - categorical_accuracy: 0.8000
Epoch 18/32
15/15 [==============================] - 8s 519ms/step - loss: 1.3406 - categorical_accuracy: 0.3333
Epoch 19/32
15/15 [==============================] - 8s 564ms/step - loss: 0.8232 - categorical_accuracy: 0.7333
Epoch 20/32
15/15 [==============================] - 8s 539ms/step - loss: 1.0908 - categorical_accuracy: 0.4667
Epoch 21/32
15/15 [==============================] - 9s 587ms/step - loss: 1.0234 - categorical_accuracy: 0.5333
Epoch 22/32
15/15 [==============================] - 8s 548ms/step - loss: 1.1348 - categorical_accuracy: 0.4000
Epoch 23/32
15/15 [==============================] - 8s 544ms/step - loss: 0.7601 - categorical_accuracy: 0.

15/15 [==============================] - 8s 499ms/step - loss: 0.5776 - categorical_accuracy: 0.8667
Epoch 32/32
15/15 [==============================] - 8s 507ms/step - loss: 1.0188 - categorical_accuracy: 0.6000
VGG took 259.2027108669281 seconds.
Epoch 1/32
15/15 [==============================] - 7s 490ms/step - loss: 0.9829 - categorical_accuracy: 0.6000
Epoch 2/32
15/15 [==============================] - 8s 494ms/step - loss: 1.0715 - categorical_accuracy: 0.5333
Epoch 3/32
15/15 [==============================] - 7s 484ms/step - loss: 0.9427 - categorical_accuracy: 0.6000
Epoch 4/32
15/15 [==============================] - 7s 490ms/step - loss: 1.1082 - categorical_accuracy: 0.4667
Epoch 5/32
15/15 [==============================] - 7s 489ms/step - loss: 1.1306 - categorical_accuracy: 0.4000
Epoch 6/32
15/15 [==============================] - 7s 493ms/step - loss: 1.0865 - categorical_accuracy: 0.4667
Epoch 7/32
15/15 [==============================] - 7s 497ms/step - loss: 0.92

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,0,2,0,2,2,2,0,2,2,2,...,1,1,2,2,2,2,1,1.0,1.0,2.0
1,0,0,0,0,0,0,0,0,0,2,...,0,2,1,2,0,2,1,1.0,NaN,NaN
2,2,2,2,2,0,2,2,2,2,2,...,1,2,1,0,0,0,2,2.0,1.0,NaN
3,0,2,0,2,2,2,2,0,2,0,...,2,2,0,2,2,0,0,0.0,2.0,NaN
4,2,2,0,0,0,2,0,2,2,2,...,2,2,2,0,0,2,2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,1,1,1,1,1,1,1,1,1,...,1,1,0,1,1,1,1,1.0,0.0,NaN
96,1,2,2,2,1,2,1,1,2,1,...,1,1,2,1,1,0,2,0.0,1.0,NaN
97,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1.0,0.0,NaN
98,1,2,1,2,1,1,2,2,1,1,...,1,1,1,1,1,1,2,1.0,1.0,NaN


In [5]:
def buildModel(shape = (224, 224, 3), output = 3):
    classifier=tf.keras.Sequential()  
    
    classifier.add(InputLayer(input_shape = (224, 224, 3)))
    classifier.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
    classifier.add(BatchNormalization())
    classifier.add(MaxPooling2D(pool_size=(3, 3)))
    classifier.add(Dropout(0.25))
        
    classifier.add(Conv2D(32, (3, 3), padding="same", activation='relu'))
    classifier.add(BatchNormalization())
    classifier.add(MaxPooling2D(pool_size=(3, 3)))
    classifier.add(Dropout(0.25))
    
    classifier.add(Conv2D(16, (3, 3), padding="same", activation='relu'))
    classifier.add(BatchNormalization())
    classifier.add(MaxPooling2D(pool_size=(3, 3)))
    classifier.add(Dropout(0.25))

    classifier.add(Flatten()) 
    classifier.add(Dense(1024,activation = 'relu'))
    classifier.add(BatchNormalization())
    classifier.add(Dropout(0.2, input_shape=(1024,)))
    classifier.add(Dense(256,activation = 'relu'))
    classifier.add(BatchNormalization())
    classifier.add(Dropout(0.2, input_shape=(256,)))

    classifier.add(Dense(output,activation = 'softmax'))

    classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
    classifier.summary()
    return classifier

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

import time

checkpoint_filepath = '/Fungos/tmp/checkpoint/SimpleNetDA/'

classes = []

for index, train in dfTrain.iterrows():
    if (index%10)==0:
        model = buildModel()
    
    train = train.dropna()
    test = dfTest.loc[index]
    test = test.dropna()
    
    xTrain, yTrain = buildSet(train)
    xTest, yTest = buildSet(test)
    
    datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.0)
    
    datagen.fit(xTrain)
    
    bs = 1
    steps = len(xTrain)//bs
    
    model_checkpoint_callback = ModelCheckpoint(
        filepath=checkpoint_filepath,
        use_multiprocessing=True,
        save_weights_only=True,
        monitor='categorical_accuracy',
        mode='max',
        save_best_only=True)
    
    start_time = time.time()
    
    history = model.fit_generator(datagen.flow(xTrain, yTrain, batch_size=bs),
                             callbacks=[model_checkpoint_callback],
                             steps_per_epoch=steps//32,
                             epochs=32)
    
    print("Simple Net took %s seconds." % ((time.time() - start_time)))

    predictions = model.predict(xTest)
    pred = np.argmax(predictions, axis=-1)
    classes.append(pred)

SimpleDA = pd.DataFrame(classes)

SimpleDA.to_csv('SimpleNetDA.csv', index=False)
SimpleDA

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 224, 224, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 74, 74, 32)        18464     
_________________________________________________________________
batch_normalization_1 (Batch (None, 74, 74, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0

/Users/joseluis/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/32
15/15 [==============================] - 2s 58ms/step - loss: 1.0722 - categorical_accuracy: 0.8232
Epoch 2/32
15/15 [==============================] - 1s 59ms/step - loss: 0.9704 - categorical_accuracy: 0.8082
Epoch 3/32
15/15 [==============================] - 1s 59ms/step - loss: 1.2239 - categorical_accuracy: 0.2278
Epoch 4/32
15/15 [==============================] - 1s 61ms/step - loss: 0.9425 - categorical_accuracy: 0.6982
Epoch 5/32
15/15 [==============================] - 1s 60ms/step - loss: 0.9340 - categorical_accuracy: 0.6846
Epoch 6/32
15/15 [==============================] - 1s 60ms/step - loss: 1.0190 - categorical_accuracy: 0.5283
Epoch 7/32
15/15 [==============================] - 1s 65ms/step - loss: 1.0381 - categorical_accuracy: 0.5001
Epoch 8/32
15/15 [==============================] - 1s 63ms/step - loss: 0.9067 - categorical_accuracy: 0.6966
Epoch 9/32
15/15 [==============================] - 1s 61ms/step - loss: 1.0085 - categorical_accuracy: 0.5603
E

15/15 [==============================] - 1s 60ms/step - loss: 0.4703 - categorical_accuracy: 0.9333
Epoch 18/32
15/15 [==============================] - 1s 61ms/step - loss: 0.5203 - categorical_accuracy: 0.8667
Epoch 19/32
15/15 [==============================] - 1s 62ms/step - loss: 0.8946 - categorical_accuracy: 0.6667
Epoch 20/32
15/15 [==============================] - 1s 60ms/step - loss: 1.1092 - categorical_accuracy: 0.5333
Epoch 21/32
15/15 [==============================] - 1s 61ms/step - loss: 0.9664 - categorical_accuracy: 0.6000
Epoch 22/32
15/15 [==============================] - 1s 61ms/step - loss: 0.9878 - categorical_accuracy: 0.6000
Epoch 23/32
15/15 [==============================] - 1s 60ms/step - loss: 0.9641 - categorical_accuracy: 0.6000
Epoch 24/32
15/15 [==============================] - 1s 60ms/step - loss: 1.1312 - categorical_accuracy: 0.4667
Epoch 25/32
15/15 [==============================] - 1s 61ms/step - loss: 1.0229 - categorical_accuracy: 0.5333
Epoc

15/15 [==============================] - 1s 59ms/step - loss: 0.9441 - categorical_accuracy: 0.6000
Epoch 2/32
15/15 [==============================] - 1s 62ms/step - loss: 0.7667 - categorical_accuracy: 0.7333
Epoch 3/32
15/15 [==============================] - 1s 61ms/step - loss: 0.8590 - categorical_accuracy: 0.6667
Epoch 4/32
15/15 [==============================] - 1s 62ms/step - loss: 0.5028 - categorical_accuracy: 0.9333
Epoch 5/32
15/15 [==============================] - 1s 60ms/step - loss: 0.5501 - categorical_accuracy: 0.8667
Epoch 6/32
15/15 [==============================] - 1s 63ms/step - loss: 0.9778 - categorical_accuracy: 0.6000
Epoch 7/32
15/15 [==============================] - 1s 62ms/step - loss: 0.8955 - categorical_accuracy: 0.6000
Epoch 8/32
15/15 [==============================] - 1s 67ms/step - loss: 0.9645 - categorical_accuracy: 0.6000
Epoch 9/32
15/15 [==============================] - 1s 68ms/step - loss: 0.8053 - categorical_accuracy: 0.7333
Epoch 10/32


...

15/15 [==============================] - 1s 59ms/step - loss: 0.7726 - categorical_accuracy: 0.7333
Epoch 18/32
15/15 [==============================] - 1s 64ms/step - loss: 1.0803 - categorical_accuracy: 0.5333
Epoch 19/32
15/15 [==============================] - 1s 58ms/step - loss: 1.1139 - categorical_accuracy: 0.4667
Epoch 20/32
15/15 [==============================] - 1s 56ms/step - loss: 1.0985 - categorical_accuracy: 0.4667
Epoch 21/32
15/15 [==============================] - 1s 56ms/step - loss: 0.9333 - categorical_accuracy: 0.6000
Epoch 22/32
15/15 [==============================] - 1s 60ms/step - loss: 0.9563 - categorical_accuracy: 0.6000
Epoch 23/32
15/15 [==============================] - 1s 63ms/step - loss: 0.7713 - categorical_accuracy: 0.8000
Epoch 24/32
15/15 [==============================] - 1s 62ms/step - loss: 0.9671 - categorical_accuracy: 0.6000
Epoch 25/32
15/15 [==============================] - 1s 68ms/step - loss: 0.8184 - categorical_accuracy: 0.7333
Epoc

15/15 [==============================] - 1s 63ms/step - loss: 0.8264 - categorical_accuracy: 0.7333
Epoch 2/32
15/15 [==============================] - 1s 62ms/step - loss: 1.0774 - categorical_accuracy: 0.4667
Epoch 3/32
15/15 [==============================] - 1s 67ms/step - loss: 0.9289 - categorical_accuracy: 0.6000
Epoch 4/32
15/15 [==============================] - 1s 63ms/step - loss: 0.7981 - categorical_accuracy: 0.7333
Epoch 5/32
15/15 [==============================] - 1s 64ms/step - loss: 1.1909 - categorical_accuracy: 0.4000
Epoch 6/32
15/15 [==============================] - 1s 64ms/step - loss: 0.9027 - categorical_accuracy: 0.6667
Epoch 7/32
15/15 [==============================] - 1s 67ms/step - loss: 1.2004 - categorical_accuracy: 0.4000
Epoch 8/32
15/15 [==============================] - 1s 65ms/step - loss: 0.8884 - categorical_accuracy: 0.6667
Epoch 9/32
15/15 [==============================] - 1s 65ms/step - loss: 1.0177 - categorical_accuracy: 0.5333
Epoch 10/32


15/15 [==============================] - 1s 61ms/step - loss: 0.7925 - categorical_accuracy: 0.7333
Epoch 24/32
15/15 [==============================] - 1s 61ms/step - loss: 1.2262 - categorical_accuracy: 0.4000
Epoch 25/32
15/15 [==============================] - 1s 60ms/step - loss: 1.0201 - categorical_accuracy: 0.5333
Epoch 26/32
15/15 [==============================] - 1s 62ms/step - loss: 0.8664 - categorical_accuracy: 0.6667
Epoch 27/32
15/15 [==============================] - 1s 61ms/step - loss: 0.7373 - categorical_accuracy: 0.8000
Epoch 28/32
15/15 [==============================] - 1s 62ms/step - loss: 0.7903 - categorical_accuracy: 0.7333
Epoch 29/32
15/15 [==============================] - 1s 60ms/step - loss: 0.7952 - categorical_accuracy: 0.7333
Epoch 30/32
15/15 [==============================] - 1s 65ms/step - loss: 0.8599 - categorical_accuracy: 0.6667
Epoch 31/32
15/15 [==============================] - 1s 62ms/step - loss: 0.7914 - categorical_accuracy: 0.7333
Epoc

15/15 [==============================] - 1s 62ms/step - loss: 0.9651 - categorical_accuracy: 0.6000
Epoch 8/32
15/15 [==============================] - 1s 61ms/step - loss: 0.8029 - categorical_accuracy: 0.7333
Epoch 9/32
15/15 [==============================] - 1s 62ms/step - loss: 0.9459 - categorical_accuracy: 0.6000
Epoch 10/32
15/15 [==============================] - 1s 60ms/step - loss: 1.0998 - categorical_accuracy: 0.4667
Epoch 11/32
15/15 [==============================] - 1s 61ms/step - loss: 0.9418 - categorical_accuracy: 0.6000
Epoch 12/32
15/15 [==============================] - 1s 60ms/step - loss: 0.9701 - categorical_accuracy: 0.5333
Epoch 13/32
15/15 [==============================] - 1s 60ms/step - loss: 0.6561 - categorical_accuracy: 0.8667
Epoch 14/32
15/15 [==============================] - 1s 62ms/step - loss: 0.7541 - categorical_accuracy: 0.7333
Epoch 15/32
15/15 [==============================] - 1s 62ms/step - loss: 1.0467 - categorical_accuracy: 0.5333
Epoch 

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,1,0,0,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0.0,0.0,0.0
1,0,0,1,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0.0,NaN,NaN
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,NaN
3,0,0,0,1,0,1,1,0,1,0,...,1,0,0,0,0,0,0,0.0,0.0,NaN
4,0,0,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,NaN
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,NaN
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,NaN
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,NaN


In [6]:
checkpoint_filepath = '/Fungos/tmp/checkpoint/SimpleNet/'

classes = []

for index, train in dfTrain.iterrows():
    if (index%10)==0:
        model = buildModel()
    
    train = train.dropna()
    test = dfTest.loc[index]
    test = test.dropna()
    
    xTrain, yTrain = buildSet(train)
    xTest, yTest = buildSet(test)
    
    bs = 1
    steps = len(xTrain)//bs
    
    model_checkpoint_callback = ModelCheckpoint(
        filepath=checkpoint_filepath,
        use_multiprocessing=True,
        save_weights_only=True,
        monitor='categorical_accuracy',
        mode='max',
        save_best_only=True)
    
    start_time = time.time()
    
    history = model.fit_generator(ImageDataGenerator().flow(xTrain, yTrain, batch_size=bs),
                             callbacks=[model_checkpoint_callback],
                             steps_per_epoch=steps//32,
                             epochs=32)
    
    print("Simple Net took %s seconds." % ((time.time() - start_time)))

    predictions = model.predict(xTest)
    pred = np.argmax(predictions, axis=-1)
    classes.append(pred)

SimpleNet = pd.DataFrame(classes)

SimpleNet.to_csv('SimpleNet.csv', index=False)
SimpleNet

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
batch_normalization_50 (Batc (None, 224, 224, 64)      256       
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 74, 74, 64)        0         
_________________________________________________________________
dropout_50 (Dropout)         (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 74, 74, 32)        18464     
_________________________________________________________________
batch_normalization_51 (Batc (None, 74, 74, 32)        128       
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 24, 24, 32)      

/Users/joseluis/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/32
15/15 [==============================] - 2s 60ms/step - loss: 1.1055 - categorical_accuracy: 0.0760
Epoch 2/32
15/15 [==============================] - 1s 60ms/step - loss: 1.0611 - categorical_accuracy: 0.7646
Epoch 3/32
15/15 [==============================] - 1s 62ms/step - loss: 1.0792 - categorical_accuracy: 0.4570
Epoch 4/32
15/15 [==============================] - 1s 60ms/step - loss: 0.9823 - categorical_accuracy: 0.7093
Epoch 5/32
15/15 [==============================] - 1s 61ms/step - loss: 0.9807 - categorical_accuracy: 0.6680
Epoch 6/32
15/15 [==============================] - 1s 61ms/step - loss: 1.0491 - categorical_accuracy: 0.5948
Epoch 7/32
15/15 [==============================] - 1s 62ms/step - loss: 1.0607 - categorical_accuracy: 0.5325
Epoch 8/32
15/15 [==============================] - 1s 61ms/step - loss: 0.7964 - categorical_accuracy: 0.8985
Epoch 9/32
15/15 [==============================] - 1s 61ms/step - loss: 0.7991 - categorical_accuracy: 0.7833
E

15/15 [==============================] - 1s 61ms/step - loss: 0.8765 - categorical_accuracy: 0.6667
Epoch 18/32
15/15 [==============================] - 1s 60ms/step - loss: 0.7795 - categorical_accuracy: 0.7333
Epoch 19/32
15/15 [==============================] - 1s 60ms/step - loss: 1.0774 - categorical_accuracy: 0.5333
Epoch 20/32
15/15 [==============================] - 1s 60ms/step - loss: 0.8675 - categorical_accuracy: 0.6667
Epoch 21/32
15/15 [==============================] - 1s 60ms/step - loss: 1.1256 - categorical_accuracy: 0.4667
Epoch 22/32
15/15 [==============================] - 1s 60ms/step - loss: 0.7236 - categorical_accuracy: 0.8000
Epoch 23/32
15/15 [==============================] - 1s 60ms/step - loss: 0.8794 - categorical_accuracy: 0.6667
Epoch 24/32
15/15 [==============================] - 1s 61ms/step - loss: 0.9539 - categorical_accuracy: 0.6000
Epoch 25/32
15/15 [==============================] - 1s 60ms/step - loss: 1.1271 - categorical_accuracy: 0.4667
Epoc

15/15 [==============================] - 1s 65ms/step - loss: 0.7674 - categorical_accuracy: 0.7333
Epoch 2/32
15/15 [==============================] - 1s 66ms/step - loss: 0.9625 - categorical_accuracy: 0.6000
Epoch 3/32
15/15 [==============================] - 1s 64ms/step - loss: 0.7889 - categorical_accuracy: 0.7333
Epoch 4/32
15/15 [==============================] - 1s 63ms/step - loss: 1.0602 - categorical_accuracy: 0.5333
Epoch 5/32
15/15 [==============================] - 1s 62ms/step - loss: 0.8778 - categorical_accuracy: 0.6667
Epoch 6/32
15/15 [==============================] - 1s 68ms/step - loss: 0.9524 - categorical_accuracy: 0.6000
Epoch 7/32
15/15 [==============================] - 1s 67ms/step - loss: 0.8719 - categorical_accuracy: 0.6667
Epoch 8/32
15/15 [==============================] - 1s 86ms/step - loss: 0.9521 - categorical_accuracy: 0.6000
Epoch 9/32
15/15 [==============================] - 1s 98ms/step - loss: 1.1082 - categorical_accuracy: 0.4667
Epoch 10/32


...

15/15 [==============================] - 1s 59ms/step - loss: 0.8141 - categorical_accuracy: 0.7333
Epoch 18/32
15/15 [==============================] - 1s 58ms/step - loss: 0.8779 - categorical_accuracy: 0.6667
Epoch 19/32
15/15 [==============================] - 1s 57ms/step - loss: 1.0312 - categorical_accuracy: 0.5333
Epoch 20/32
15/15 [==============================] - 1s 62ms/step - loss: 0.9551 - categorical_accuracy: 0.6000
Epoch 21/32
15/15 [==============================] - 1s 60ms/step - loss: 0.8619 - categorical_accuracy: 0.6667
Epoch 22/32
15/15 [==============================] - 1s 60ms/step - loss: 0.9572 - categorical_accuracy: 0.6000
Epoch 23/32
15/15 [==============================] - 1s 58ms/step - loss: 0.8004 - categorical_accuracy: 0.7333
Epoch 24/32
15/15 [==============================] - 1s 58ms/step - loss: 1.2550 - categorical_accuracy: 0.3333
Epoch 25/32
15/15 [==============================] - 1s 58ms/step - loss: 0.8808 - categorical_accuracy: 0.6667
Epoc

15/15 [==============================] - 1s 60ms/step - loss: 0.7941 - categorical_accuracy: 0.7333
Epoch 2/32
15/15 [==============================] - 1s 57ms/step - loss: 0.9289 - categorical_accuracy: 0.6000
Epoch 3/32
15/15 [==============================] - 1s 58ms/step - loss: 0.5947 - categorical_accuracy: 0.8667
Epoch 4/32
15/15 [==============================] - 1s 57ms/step - loss: 1.0896 - categorical_accuracy: 0.5333
Epoch 5/32
15/15 [==============================] - 1s 57ms/step - loss: 0.7486 - categorical_accuracy: 0.7333
Epoch 6/32
15/15 [==============================] - 1s 57ms/step - loss: 0.9562 - categorical_accuracy: 0.6000
Epoch 7/32
15/15 [==============================] - 1s 57ms/step - loss: 0.9485 - categorical_accuracy: 0.6000
Epoch 8/32
15/15 [==============================] - 1s 57ms/step - loss: 1.0275 - categorical_accuracy: 0.5333
Epoch 9/32
15/15 [==============================] - 1s 57ms/step - loss: 1.0076 - categorical_accuracy: 0.6000
Epoch 10/32


15/15 [==============================] - 1s 57ms/step - loss: 0.9949 - categorical_accuracy: 0.5333
Epoch 24/32
15/15 [==============================] - 1s 58ms/step - loss: 1.0219 - categorical_accuracy: 0.6000
Epoch 25/32
15/15 [==============================] - 1s 58ms/step - loss: 0.9672 - categorical_accuracy: 0.6000
Epoch 26/32
15/15 [==============================] - 1s 58ms/step - loss: 1.1680 - categorical_accuracy: 0.4000
Epoch 27/32
15/15 [==============================] - 1s 57ms/step - loss: 1.0077 - categorical_accuracy: 0.5333
Epoch 28/32
15/15 [==============================] - 1s 60ms/step - loss: 1.1891 - categorical_accuracy: 0.3333
Epoch 29/32
15/15 [==============================] - 1s 58ms/step - loss: 0.8945 - categorical_accuracy: 0.6667
Epoch 30/32
15/15 [==============================] - 1s 58ms/step - loss: 0.8638 - categorical_accuracy: 0.7333
Epoch 31/32
15/15 [==============================] - 1s 58ms/step - loss: 0.8126 - categorical_accuracy: 0.7333
Epoc

15/15 [==============================] - 1s 59ms/step - loss: 0.6718 - categorical_accuracy: 0.8000
Epoch 8/32
15/15 [==============================] - 1s 63ms/step - loss: 0.7577 - categorical_accuracy: 0.7333
Epoch 9/32
15/15 [==============================] - 1s 67ms/step - loss: 1.0863 - categorical_accuracy: 0.5333
Epoch 10/32
15/15 [==============================] - 1s 86ms/step - loss: 0.7910 - categorical_accuracy: 0.7333
Epoch 11/32
15/15 [==============================] - 1s 59ms/step - loss: 1.0394 - categorical_accuracy: 0.5333
Epoch 12/32
15/15 [==============================] - 1s 61ms/step - loss: 0.9639 - categorical_accuracy: 0.6000
Epoch 13/32
15/15 [==============================] - 1s 64ms/step - loss: 0.8024 - categorical_accuracy: 0.7333
Epoch 14/32
15/15 [==============================] - 1s 75ms/step - loss: 1.1041 - categorical_accuracy: 0.4667
Epoch 15/32
15/15 [==============================] - 1s 66ms/step - loss: 1.3072 - categorical_accuracy: 0.3333
Epoch 

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,1,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1.0,1.0,1.0
1,1,1,0,1,1,1,0,1,1,1,...,1,1,1,1,1,1,1,1.0,NaN,NaN
2,0,1,1,0,0,0,0,1,0,0,...,1,1,1,1,1,1,1,1.0,1.0,NaN
3,0,1,0,0,1,1,1,1,1,0,...,1,1,1,1,1,1,1,1.0,0.0,NaN
4,0,1,1,1,1,2,1,0,0,1,...,1,0,1,1,1,1,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,2,2,2,1,2,2,0,2,2,...,1,2,2,2,1,2,2,2.0,1.0,NaN
96,2,2,2,2,1,1,1,1,2,2,...,2,2,1,1,1,2,2,2.0,2.0,NaN
97,1,2,2,1,0,2,2,2,2,1,...,1,1,1,1,1,1,1,1.0,1.0,NaN
98,2,2,2,2,2,2,2,1,1,2,...,2,1,1,1,1,2,1,1.0,1.0,NaN
